In [1]:
from typing import List, Union, Tuple
from functools import reduce
from torch_geometric import nn as pnn
import numpy as np
from rdkit import Chem
import torch
import torch.nn as nn
from torch_geometric.data import Data, Batch
from torch_scatter import scatter_sum, scatter_mean
from torch_geometric.nn.norm.batch_norm import BatchNorm
import time
from torch_geometric.nn.models import MLP,GCN,GAT,GraphSAGE,GIN,BasicGNN
import networkx as nx
import math
from torch.nn.utils.rnn import pad_sequence

from typing import Any, List, Optional, Tuple,Final
from torch import Tensor
from torch_geometric.utils import cumsum, to_dense_batch, scatter
from torch_geometric.typing import OptTensor
import matplotlib.pyplot as plt
from torch_geometric.nn import SAGPooling
from typing import Callable, Optional, Union
import torch

from torch import Tensor
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.inits import reset
from torch_geometric.typing import (
    Adj,
    OptPairTensor,
    OptTensor,
    Size,
    SparseTensor,
)
from torch_geometric.utils import spmm
from torch_geometric.nn.dense import DenseGINConv
import torch.nn.functional as F


/home/jiaopanyu/.conda/envs/chemprop/lib/python3.8/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.27' not found (required by /home/jiaopanyu/.conda/envs/chemprop/lib/python3.8/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/jiaopanyu/.conda/envs/chemprop/lib/python3.8/site-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /home/jiaopanyu/.conda/envs/chemprop/lib/python3.8/site-packages/torch_spline_conv/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(


##### 准备数据

In [ ]:
f_atoms = torch.load('/home/jiaopanyu/save_components/f_atoms.pth')

In [ ]:
f_atoms.shape,f_atoms

In [ ]:
f_bonds = torch.load('/home/jiaopanyu/save_components/f_bonds.pth')

In [ ]:
f_bonds.shape

In [ ]:
f_map = torch.load('/home/jiaopanyu/save_components/f_map.pth')

In [ ]:
len(f_map),f_map

In [ ]:
f_xyz = torch.load('/home/jiaopanyu/save_components/f_xyz.pth')

In [ ]:
f_xyz.shape,f_xyz

In [ ]:
a_scope = torch.load('/home/jiaopanyu/save_components/a_scope.pth')
b_scope = torch.load('/home/jiaopanyu/save_components/b_scope.pth')

In [ ]:
a2b = torch.load('/home/jiaopanyu/save_components/a2b.pth')
b2a = torch.load('/home/jiaopanyu/save_components/b2a.pth')

In [ ]:
b2revb = torch.load('/home/jiaopanyu/save_components/b2revb.pth')
b2revb

In [ ]:
def get_unit_sequence(input_dim, output_dim, n_hidden):
    """
    Smoothly decay the number of hidden units in each layer.
    Start from 'input_dim' and end with 'output_dim'.
  
    Examples:
    get_unit_sequence(1,1024,4) = [1, 4, 16, 64, 256, 1024]
    get_unit_sequence(1024,1,4) = [1024, 256, 64, 16, 4, 1]
    """
    reverse = False
    if input_dim > output_dim:
        reverse = True
        input_dim,output_dim = output_dim,input_dim

    diff = abs(output_dim.bit_length() - input_dim.bit_length())
    increment = diff // (n_hidden+1)

    sequence = [input_dim] + [0] * (n_hidden) + [output_dim]

    for idx in range(n_hidden // 2):
        sequence[idx+1] = 2 ** ((sequence[idx]).bit_length() + increment-1)
        sequence[-2-idx] = 2 ** ((sequence[-1-idx]-1).bit_length() - increment)

    if n_hidden%2 == 1:
        sequence[n_hidden // 2 + 1] = (sequence[n_hidden // 2] + sequence[n_hidden // 2+2])//2

    if reverse: 
        sequence.reverse()

    return sequence

In [ ]:
class hidden(nn.Module):
    def __init__(self, size_in, size_out, activation):
        super().__init__()
        self.linear = nn.Linear(size_in, size_out)
        self.activation = activation

    def forward(self, x):
        return self.activation(self.linear(x))

In [ ]:
class ffn(nn.Module):
    """
    A Feed-Forward neural Network that uses DenseHidden layers
    """

    def __init__(self, input_dim, output_dim, capacity, activation):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.activation = activation
        self.layers = nn.ModuleList()
        self.unit_sequence = get_unit_sequence(
            input_dim, output_dim, capacity
        )
        # set up hidden layers
        for ind, n_units in enumerate(self.unit_sequence[:-1]):
            size_out_ = self.unit_sequence[ind + 1]
            self.layers.append(
                hidden(
                    size_in=n_units,
                    size_out=size_out_,
                    activation=self.activation,
                )
            )

    def forward(self, x):
        """
        Compute the forward pass of this model
        """
        for layer in self.layers:
            x = layer(x)

        return x

##### model:polygin

In [2]:
from chempropfix.args import TrainArgs
from chempropfix.nn_utils import get_activation_function
from chempropfix.models.mpn import GAIN,MPNEncoder,torch_adj_helper,mol_graph2data
from chempropfix.features import BatchMolGraph, get_atom_fdim, get_bond_fdim, mol2graph
from chempropfix.models.model import MoleculeModel
from chempropfix.models.egnn import E_GCL_mask,EGNN
from chempropfix.models.gcl import E_GCL, unsorted_segment_sum

/home/jiaopanyu/.conda/envs/chemprop/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class POLYGINConv(MessagePassing):
    r"""The graph isomorphism operator from the `"How Powerful are
    Graph Neural Networks?" <https://arxiv.org/abs/1810.00826>`_ paper

    .. math::
        \mathbf{x}^{\prime}_i = h_{\mathbf{\Theta}} \left( (1 + \epsilon) \cdot
        \mathbf{x}_i + \sum_{j \in \mathcal{N}(i)} \mathbf{x}_j \right)

    or

    .. math::
        \mathbf{X}^{\prime} = h_{\mathbf{\Theta}} \left( \left( \mathbf{A} +
        (1 + \epsilon) \cdot \mathbf{I} \right) \cdot \mathbf{X} \right),

    here :math:`h_{\mathbf{\Theta}}` denotes a neural network, *.i.e.* an MLP.

    Args:
        nn (torch.nn.Module): A neural network :math:`h_{\mathbf{\Theta}}` that
            maps node features :obj:`x` of shape :obj:`[-1, in_channels]` to
            shape :obj:`[-1, out_channels]`, *e.g.*, defined by
            :class:`torch.nn.Sequential`.
        eps (float, optional): (Initial) :math:`\epsilon`-value.
            (default: :obj:`0.`)
        train_eps (bool, optional): If set to :obj:`True`, :math:`\epsilon`
            will be a trainable parameter. (default: :obj:`False`)
        **kwargs (optional): Additional arguments of
            :class:`torch_geometric.nn.conv.MessagePassing`.

    Shapes:
        - **input:**
          node features :math:`(|\mathcal{V}|, F_{in})` or
          :math:`((|\mathcal{V_s}|, F_{s}), (|\mathcal{V_t}|, F_{t}))`
          if bipartite,
          edge indices :math:`(2, |\mathcal{E}|)`
        - **output:** node features :math:`(|\mathcal{V}|, F_{out})` or
          :math:`(|\mathcal{V}_t|, F_{out})` if bipartite
    """
    def __init__(self,in_channels,out_channels, nn: Callable, eps: float = 0., train_eps: bool = False, node_hidden: Optional[int] = None, edge_hidden: Optional[int] = None,activation:Callable=None,
                 **kwargs):
        kwargs.setdefault('aggr', 'add')
        super().__init__(**kwargs)
        self.edge_mapper = ffn(edge_hidden, edge_hidden,capacity=1,activation=activation)
        self.residual_mapping = ffn(in_channels+edge_hidden, in_channels,capacity=1,activation=activation)
        # self.node_mapper = ffn(in_channels, in_channels,capacity=1,activation=activation)
        self.nn = nn
        self.initial_eps = eps
        if train_eps:
            self.eps = torch.nn.Parameter(torch.empty(1))
        else:
            self.register_buffer('eps', torch.empty(1))
        self.reset_parameters()

    def reset_parameters(self):
        super().reset_parameters()
        reset(self.nn)
        self.eps.data.fill_(self.initial_eps)

    def forward(self, x: Union[Tensor, OptPairTensor], edge_index: Adj,edge_attr: Union[Tensor, OptPairTensor],
                size: Size = None) -> Tensor:
      
        if isinstance(x, Tensor):
            x: OptPairTensor = (x, x)

        out = self.propagate(edge_index,size=size,x=x,edge_attr=edge_attr)

        x_r = x[1]
        if x_r is not None:
            out = out + (1 + self.eps) * x_r
        return self.nn(out)

    def message(self, x_j: Tensor, edge_attr: Tensor) -> Tensor:
        # x_j = self.node_mapper(x_j)
        edge_attr = self.edge_mapper(edge_attr)
        return self.residual_mapping(torch.concat((x_j,edge_attr),dim=-1))

    # def message_and_aggregate(self, adj_t: SparseTensor,
    #                           x: OptPairTensor) -> Tensor:
    #     if isinstance(adj_t, SparseTensor):
    #         adj_t = adj_t.set_value(None, layout=None)
    #     return spmm(adj_t, x[0], reduce=self.aggr)

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}(nn={self.nn})'

In [4]:
class POLYGIN(BasicGNN):
    r"""The Graph Neural Network from the `"How Powerful are Graph Neural
    Networks?" <https://arxiv.org/abs/1810.00826>`_ paper, using the
    :class:`~torch_geometric.nn.GINConv` operator for message passing.

    Args:
        in_channels (int): Size of each input sample.
        hidden_channels (int): Size of each hidden sample.
        num_layers (int): Number of message passing layers.
        out_channels (int, optional): If not set to :obj:`None`, will apply a
            final linear transformation to convert hidden node embeddings to
            output size :obj:`out_channels`. (default: :obj:`None`)
        dropout (float, optional): Dropout probability. (default: :obj:`0.`)
        act (str or Callable, optional): The non-linear activation function to
            use. (default: :obj:`"relu"`)
        act_first (bool, optional): If set to :obj:`True`, activation is
            applied before normalization. (default: :obj:`False`)
        act_kwargs (Dict[str, Any], optional): Arguments passed to the
            respective activation function defined by :obj:`act`.
            (default: :obj:`None`)
        norm (str or Callable, optional): The normalization function to
            use. (default: :obj:`None`)
        norm_kwargs (Dict[str, Any], optional): Arguments passed to the
            respective normalization function defined by :obj:`norm`.
            (default: :obj:`None`)
        jk (str, optional): The Jumping Knowledge mode. If specified, the model
            will additionally apply a final linear transformation to transform
            node embeddings to the expected output feature dimensionality.
            (:obj:`None`, :obj:`"last"`, :obj:`"cat"`, :obj:`"max"`,
            :obj:`"lstm"`). (default: :obj:`None`)
        **kwargs (optional): Additional arguments of
            :class:`torch_geometric.nn.conv.GINConv`.
    """
    supports_edge_weight: Final[bool] = False
    supports_edge_attr: Final[bool] = True
    supports_norm_batch: Final[bool]

    def init_conv(self, in_channels: int, out_channels: int,
                  **kwargs) -> MessagePassing:
        mlp = MLP(
            [in_channels,in_channels, out_channels],
            act=self.act,
            act_first=self.act_first,
            norm=self.norm,
            norm_kwargs=self.norm_kwargs,
        )
        return POLYGINConv(in_channels,out_channels,mlp, **kwargs)


In [7]:
def repeat_interleave(
    repeats: List[int],
    device: Optional[torch.device] = None,
) -> Tensor:
    outs = [torch.full((n, ), i, device=device) for i, n in enumerate(repeats)]
    return torch.cat(outs, dim=0)

In [9]:
class mol_graph2data(nn.Module):
    def __init__(self,args):
        super(mol_graph2data,self).__init__()
        self.args = args

    def forward(self,
        mol_graph: BatchMolGraph,
        atom_descriptors_batch: List[np.ndarray] = None) -> torch.FloatTensor:
        
    
        f_atoms, f_bonds,a2b, b2a, b2revb, a_scope, b_scope,f_map,f_xyz = mol_graph.get_components()
        print(f_atoms.shape)
        print(f_bonds.shape)
        print(a2b.shape)
        print(b2a.shape)
        print(b2revb.shape)
        print(a_scope.shape)
        print(b_scope.shape)
        print(f_map.shape)
        print(f_xyz.shape)


        repeats = [i[1] for i in a_scope]  # 原子数
        batch = repeat_interleave(repeats, device=self.args.device)
        ptr = cumsum(torch.tensor(repeats, device=self.args.device))
        b2revb_individual = []
        edge_index = [[],[]]
        for idx, (a_scope_i, b_scope_i) in enumerate(zip(a_scope, b_scope)):
            b2a_i = b2a[b_scope_i[0]:b_scope_i[0] + b_scope_i[1]]
            b2revb_i = b2revb[b_scope_i[0]:b_scope_i[0] + b_scope_i[1]]
            edge_index[0].extend((b2a_i).tolist())
            edge_index[1].extend((b2a_i[b2revb_i - b_scope_i[0]]).tolist())
            # 如果使用pyG，那么就不需要b2revb_individual
            # b2revb_individual.append(b2revb[b_scope_i[0]:b_scope_i[0]+b_scope_i[1]]-b_scope_i[0])
            # b2revb_individual = torch.cat(b2revb_individual,dim=0)
        edge_index = torch.tensor(edge_index,dtype=torch.long,device=self.args.device)-1    #因为chemprop内部的设置，这里必须-1
        return f_atoms[1:,:].to(self.args.device),\
               edge_index, \
               f_bonds[1:,:].to(self.args.device),\
               b2revb_individual, \
               batch, ptr

In [5]:
class pyG_helper(nn.Module):
    def __init__(self, args: TrainArgs, atom_fdim: int, bond_fdim: int):

        super(pyG_helper, self).__init__()
        # 创建数据格式：mol_graph2data
        self.mol_graph2data_layer = mol_graph2data(args)
        self.args = args
        self.encoder_type = args.encoder_type

        if args.encoder_type in ["gcn","gcn_attn","gcn_pe"]:
            self.mpn = GCN(in_channels=atom_fdim,hidden_channels=args.hidden_size,num_layers=args.depth,dropout=args.gnn_dropout,activation=get_activation_function(args.activation))
        elif args.encoder_type in ["gat","gat_attn","gat_pe"]:
            self.mpn = GAT(in_channels=atom_fdim,hidden_channels=args.hidden_size,num_layers=args.depth,dropout=args.gnn_dropout,activation=get_activation_function(args.activation))
        elif args.encoder_type in ["graphsage","graphsage_attn","graphsage_pe"]:
            self.mpn = GraphSAGE(in_channels=atom_fdim,hidden_channels=args.hidden_size,num_layers=args.depth,dropout=args.gnn_dropout,activation=get_activation_function(args.activation))
        elif args.encoder_type in ["gin","gin_attn","gin_pe"]:
            self.mpn = GIN(in_channels=atom_fdim,hidden_channels=args.hidden_size,num_layers=args.depth,dropout=args.gnn_dropout,act=get_activation_function(args.activation))
        elif args.encoder_type in ["polygnn", "polygnn_attn", "polygnn_pe"]:
            hps={
                "readout_dim" : args.hidden_size,
                "depth" : args.depth,
                "activation" : get_activation_function(args.activation),
                "ffn_capacity": 0,
                "dropout":args.gnn_dropout,
                "visual": 21
            }
            # self.mpn = polygnn_mp(node_size= atom_fdim, edge_size= bond_fdim,hps=hps,normalize_embedding=True)
        elif args.encoder_type in ["gain", "gain_attn", "gain_pe"]:
            self.mpn = GAIN(args, atom_fdim, bond_fdim)
        #elif args.encoder_type in ['graphformer']:
            #self.mpn = graphformerEncoder(args, atom_fdim, bond_fdim)
        elif args.encoder_type in ["polygin", "polygin_attn"]:
            # self.node_mapper = nn.Linear(atom_fdim,args.hidden_size)
            self.mpn = POLYGIN(in_channels=atom_fdim,hidden_channels=args.hidden_size,num_layers=args.depth,dropout=args.gnn_dropout,activation=get_activation_function(args.activation),edge_hidden=bond_fdim,node_hidden=args.hidden_size)
        #self.with_attn = args.with_attn
        #if self.with_attn:
        #    self.attention_encoder = AttnEncoderXL(args,bond_fdim)
        # self.layer_norm  = nn.LayerNorm(args.hidden_size, eps=1e-6,elementwise_affine=True)

    def forward(self,
                mol_graph: BatchMolGraph,
                atom_descriptors_batch: List[np.ndarray] = None) -> torch.FloatTensor:
        """
        f_atoms[1:,:].to(self.args.device),\
               edge_index, \
               f_bonds[1:,:].to(self.args.device),\
               b2revb_individual, \
               batch, ptr
        """
        # 做数据转化
        x, edge_index, edge_attr, b2revb_individual, batch, ptr = self.mol_graph2data_layer(mol_graph) 
        
        if self.args.encoder_type in ["polygin", "polygin_attn", "polygin_pe"]:
            # x = self.node_mapper(x)
            poly_vec = self.mpn(x=x, edge_index=edge_index, edge_attr=edge_attr, batch=batch)
            # poly_vec = w_atoms.view(w_atoms.shape[0], 1) * poly_vec
            # poly_vec = scatter_sum(poly_vec, batch, dim=0)
            # w_atoms = scatter_sum(w_atoms, batch, dim=0)
            # poly_vec = poly_vec / w_atoms.view(w_atoms.shape[0], 1)
        #elif self.encoder_type in ["polygnn", "polygnn_attn", "polygnn_pe"]:
            #poly_vec = self.mpn(x=x, edge_index=edge_index, edge_attr=edge_attr,batch=batch)
        #elif self.encoder_type in ["graphformer"]:
             #poly_vec = self.mpn(x, edge_index, edge_attr,ptr)
        else:
            poly_vec = self.mpn(x=x, edge_index=edge_index, edge_attr=edge_attr,batch=batch)
           
        poly_vec = scatter_sum(poly_vec, batch, dim=0)
        #if self.args.visual == 21:
            #return poly_vec
        
        """
        if self.with_attn:
            lengths = ptr[1:] - ptr[:-1]
            dense_batch, mask = to_dense_batch(torch.arange(poly_vec.size(0),device=self.args.device), batch=batch)
            mask_matrix = ~(mask.unsqueeze(2) & mask.unsqueeze(1))        # [batch_size, max_size] => [batch_size,max_size,max_size]
            node_count = torch.bincount(batch)
            node_features_split = torch.split(poly_vec, node_count.tolist())
            max_length = dense_batch.size(1)
            padded_tensor = torch.stack([torch.cat([feat, torch.zeros(max_length - feat.size(0), feat.size(1),device=self.args.device)]) for feat in node_features_split])

            atom_message, attns = self.attention_encoder(poly_vec,padded_tensor,mask_matrix,edge_index,edge_attr,ptr,batch,distances,node_paths,edge_paths)
            # atom_message = torch.cat([atom_message[i, :a,:] for i,a in enumerate(lengths)], dim=0)
            attns = torch.sum(attns,dim=1)
            if self.args.visual:
                return attns
                # torch.save(attns, '/home/chenlidong/polyAttn/notebooks/tensor.pt')
                # print(attns.cpu().detach().numpy())
            if not self.args.svd:
                attns = attns.masked_fill(mask_matrix, 0)
                attns = torch.mean(attns,dim=1)
                atom_message = atom_message * attns.unsqueeze(-1)       # 需要再次加上w信息吗？
                poly_vec = torch.sum(atom_message,dim=1) / torch.sum(attns,dim=1).unsqueeze(1)
            else:
                # pdb.set_trace()
                atom_message[~mask.unsqueeze(-1).expand_as(atom_message)] = 0
                U, S, V = torch.svd(atom_message, some=True, compute_uv=True)
                poly_vec = torch.sum(V.transpose(-1,-2)[:,:10],dim=1)

        else:
        """

        return poly_vec


In [6]:
class MPN(nn.Module):
    """An :class:`MPN` is a wrapper around :class:`MPNEncoder` which featurizes input as needed."""

    def __init__(self,
                 args: TrainArgs,
                 atom_fdim: int = None,
                 bond_fdim: int = None):
        """
        :param args: A :class:`~chemprop.args.TrainArgs` object containing model arguments.
        :param atom_fdim: Atom feature vector dimension.
        :param bond_fdim: Bond feature vector dimension.
        """
        super(MPN, self).__init__()
        self.atom_fdim = atom_fdim or get_atom_fdim(overwrite_default_atom=args.overwrite_default_atom_features)
        
        self.bond_fdim = bond_fdim or get_bond_fdim(overwrite_default_atom=args.overwrite_default_atom_features,
                                                    overwrite_default_bond=args.overwrite_default_bond_features,
                                                    atom_messages=args.atom_messages)

        self.features_only = args.features_only
        self.use_input_features = args.use_input_features
        self.device = args.device
        self.atom_descriptors = args.atom_descriptors
        self.overwrite_default_atom_features = args.overwrite_default_atom_features
        self.overwrite_default_bond_features = args.overwrite_default_bond_features

        if self.features_only:
            return

        if args.encoder_type in ["wDMPNN","wDMPNN_origin_attn","wDMPNN_attn","wDMPNN_pe"]:
            encoder = MPNEncoder
        elif "adj" in args.encoder_type:
            encoder = torch_adj_helper
        else:
            encoder = pyG_helper 

        if args.mpn_shared:
            self.encoder = nn.ModuleList([encoder(args, self.atom_fdim, self.bond_fdim)] * args.number_of_molecules)
        else:
            self.encoder = nn.ModuleList([encoder(args, self.atom_fdim, self.bond_fdim)
                                          for _ in range(args.number_of_molecules)])
# 怎么最快的方式拿到数据
    def forward(self,
                batch: Union[List[List[str]], List[List[Chem.Mol]], List[List[Tuple[Chem.Mol, Chem.Mol]]], List[BatchMolGraph]],
                features_batch: List[np.ndarray] = None,
                atom_descriptors_batch: List[np.ndarray] = None,
                atom_features_batch: List[np.ndarray] = None,
                bond_features_batch: List[np.ndarray] = None,
                # start_time=None,
                # logger=None
                ) -> torch.FloatTensor:

        """
        Encodes a batch of molecules.

        :param batch: A list of list of SMILES, a list of list of RDKit molecules, or a
                      list of :class:`~chemprop.features.featurization.BatchMolGraph`.
                      The outer list or BatchMolGraph is of length :code:`num_molecules` (number of datapoints in batch),
                      the inner list is of length :code:`number_of_molecules` (number of molecules per datapoint).
        :param features_batch: A list of numpy arrays containing additional features.
        :param atom_descriptors_batch: A list of numpy arrays containing additional atom descriptors.
        :param atom_features_batch: A list of numpy arrays containing additional atom features.
        :param bond_features_batch: A list of numpy arrays containing additional bond features.
        :return: A PyTorch tensor of shape :code:`(num_molecules, hidden_size)` containing the encoding of each molecule.
        """
        # debug = logger.debug if logger is not None else print
        if type(batch[0]) != BatchMolGraph:
            # Group first molecules, second molecules, etc for mol2graph
            batch = [[mols[i] for mols in batch] for i in range(len(batch[0]))]

            # TODO: handle atom_descriptors_batch with multiple molecules per input
            if self.atom_descriptors == 'feature':
                if len(batch) > 1:
                    raise NotImplementedError('Atom/bond descriptors are currently only supported with one molecule '
                                              'per input (i.e., number_of_molecules = 1).')

                batch = [
                    mol2graph(
                        mols=b,
                        atom_features_batch=atom_features_batch,
                        bond_features_batch=bond_features_batch,
                        overwrite_default_atom_features=self.overwrite_default_atom_features,
                        overwrite_default_bond_features=self.overwrite_default_bond_features
                    )
                    for b in batch
                ]
            elif bond_features_batch is not None:
                if len(batch) > 1:
                    raise NotImplementedError('Atom/bond descriptors are currently only supported with one molecule '
                                              'per input (i.e., number_of_molecules = 1).')

                batch = [
                    mol2graph(
                        mols=b,
                        bond_features_batch=bond_features_batch,
                        overwrite_default_atom_features=self.overwrite_default_atom_features,
                        overwrite_default_bond_features=self.overwrite_default_bond_features
                    )
                    for b in batch
                ]
            else:
                batch = [mol2graph(b) for b in batch]
        if self.use_input_features:
            features_batch = torch.from_numpy(np.stack(features_batch)).float().to(self.device)

            if self.features_only:
                return features_batch

        if self.atom_descriptors == 'descriptor':
            if len(batch) > 1:
                raise NotImplementedError('Atom descriptors are currently only supported with one molecule '
                                          'per input (i.e., number_of_molecules = 1).')

            encodings = [enc(ba, atom_descriptors_batch) for enc, ba in zip(self.encoder, batch)]
        else:
            
            
            encodings = [enc(ba) for enc, ba in zip(self.encoder, batch)]
            
            # 使用EGNN

            

        output = reduce(lambda x, y: torch.cat((x, y), dim=1), encodings)

        if self.use_input_features:
            if len(features_batch.shape) == 1:
                features_batch = features_batch.view(1, -1)
            output = torch.cat([output, features_batch], dim=1)
        return output

In [ ]:
model = MoleculeModel(args)
model.train()
preds = model(
        mol_batch,
        features_batch,
        atom_descriptors_batch,
        atom_features_batch,
        bond_descriptors_batch,
        bond_features_batch,
        constraints_batch,
        bond_types_batch,
)

In [1]:
import os

In [3]:
os.path.join('pksgs', str(1),'args.json')

'pksgs/1/args.json'